In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import fnmatch
import seaborn as sns

In [ ]:
dataframe = pd.read_csv('full_scaffold_dataset.csv')
dataframe.sample(10)

In [ ]:
label_counts = dataframe['Class'].value_counts()[:15]
plt.figure(figsize=(11,8))
#fig = plt.subplots(1,1,figsize = (11, 8))
ax1 = sns.barplot(x=np.arange(len(label_counts))+0.5, y=label_counts)
#ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts)))
_ = ax1.set_xticklabels(label_counts.index)
plt.savefig('classes.png')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (128, 128)
datagen = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=5, 
                              shear_range = 0.1,
                              fill_mode = 'reflect',
                              zoom_range=0.15)

In [ ]:

train_generator = datagen.flow_from_directory(
        'apple-diseases/train',
        target_size=IMG_SIZE,
        color_mode = 'rgb',
        batch_size=32,
        class_mode='categorical')


x_val, y_val = next(datagen.flow_from_directory(
        'apple-diseases/validation',
        target_size=IMG_SIZE,
        color_mode = 'rgb',
        batch_size=180,
        class_mode='categorical')) # one big batch


x_test, y_test = next(datagen.flow_from_directory(
        'apple-diseases/test',
        target_size=IMG_SIZE,
        color_mode = 'rgb',
        batch_size=180,
        class_mode='categorical')) # one big batch

In [ ]:
num_classes = 4

from keras.applications.mobilenet import MobileNet 
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers import Conv2D,  SeparableConv2D
from keras.models import Sequential
base_mobilenet_model = MobileNet(input_shape =  x_val.shape[1:], 
                                 include_top = False, weights = '../input/keras-pretrained-models-no-top/mobilenet_1_0_128_tf_no_top.h5')
model = Sequential()
model.add(base_mobilenet_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                           metrics = ['categorical_accuracy', 'mae'])
model.summary()


In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('apple_diseases')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
callbacks_list = [checkpoint, early]

In [ ]:
#First Round 
model.fit_generator(train_generator, 
                    steps_per_epoch=100, 
                    validation_data = (x_val, y_val), 
                    epochs = 1, 
                    callbacks = callbacks_list)

# Serialize model to JSON
from keras.models import model_from_json
model_json = model.to_json()
with open(r'tomato_disease.json', "w") as json_file:
    json_file.write(model_json)

In [ ]:
scores = model.evaluate(x_val, y_val)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("val_loss:", scores[0])
print("val_mean_absolute_error:", scores[2])

In [ ]:
# Continued Training
#core_idg.flow(x_train, y_train, batch_size=200)
model.fit_generator(train_generator, 
                    steps_per_epoch=100, 
                    validation_data = (x_val, y_val), 
                    epochs = 12, 
                    callbacks = callbacks_list)

In [ ]:
# load the best weights
model.load_weights(weight_path)
#pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)
scores = model.evaluate(x_val, y_val)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("val_loss:", scores[0])
print("val_mean_absolute_error:", scores[2])

In [ ]:
pred_Y = model.predict(x_val, batch_size = 32, verbose = True)
print(pred_Y[:15])

In [ ]:
print(y_val[:15])

In [ ]:
y_test.shape

In [ ]:
pred_Y.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
class_labels = LabelEncoder()
prediction_le = class_labels.fit_transform(pred_Y)

In [ ]:
from sklearn.metrics import roc_curve, auc
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_val[:, i], pred_Y[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_val.ravel(), pred_Y.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


In [ ]:
from scipy import interp
from itertools import cycle

lw = 2

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot all ROC curves
plt.figure(figsize=(11,8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('roc_all.png')
    

In [ ]:
# load the best weights
model.load_weights(weight_path)
#pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("val_loss:", scores[0])
print("val_mean_absolute_error:", scores[2])

In [ ]:
pred_Y = model.predict(x_test, batch_size = 32, verbose = True)
print(pred_Y[:15])


In [ ]:
print(y_test[:15])

In [ ]:
from sklearn.metrics import roc_curve, auc
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred_Y[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), pred_Y.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])




from scipy import interp
from itertools import cycle

lw = 2

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot all ROC curves
plt.figure(figsize=(11,8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('roc_test_all.png')
    